<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [1]:
!pip install -Uqqq rich openai==0.27.2 tiktoken wandb langchain unstructured tabulate pdf2image chromadb

In [10]:
%pip install -U langchain-community
%pip install "unstructured[md]"

Note: you may need to restart the kernel to use updated packages.
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [3]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [8]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "daphane"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [5]:
MODEL_NAME = "text-davinci-003"
# MODEL_NAME = "gpt-4"

Let's grab some sample data, here we will download a folder containing some markdown files from our docs.

In [ ]:
!git clone https://github.com/wandb/edu.git

In [11]:
from langchain.document_loaders import DirectoryLoader
caminho_documentos = "../documentos_base/"

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files(caminho_documentos)
len(documents)

1

In [12]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [13]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[14811]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [14]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(56, 403)

let's look at the first section

In [15]:
Markdown(document_sections[0].page_content)

LEI Nº 11.340, DE 7 DE AGOSTO DE 2006                                                                              

Cria mecanismos para coibir a violência doméstica e familiar contra a mulher, nos termos do § 8º do art. 226 da    
Constituição Federal, da Convenção sobre a Eliminação de Todas as Formas de Discriminação contra as Mulheres e da  
Convenção Interamericana para Prevenir, Punir e Erradicar a Violência contra a Mulher; dispõe sobre a criação dos  
Juizados de Violência Doméstica e Familiar contra a Mulher; altera o Código de Processo Penal, o Código Penal e a  
Lei de Execução Penal; e dá outras providências.                                                                   

O PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:                

TÍTULO I                                                                                                           

DISPOSIÇÕES PRELIMINARES

## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 

In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

/tmp/ipykernel_22952/2891843054.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [17]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [18]:
query = "Quais os tipos de violência contra a mulher?"
docs = retriever.get_relevant_documents(query)

/tmp/ipykernel_22952/2151988766.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [23]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])
    print(f"Início do contexto: =======\n {doc.page_content}\nFim do contexto ======\n")

../documentos_base/lei-maria-da-penha.md
Início do contexto: =======
 Art. 7º São formas de violência doméstica e familiar contra a mulher, entre outras: I - a violência física, entendida como qualquer conduta que ofenda sua integridade ou saúde corporal; II - a violência psicológica, entendida como qualquer conduta que lhe cause dano emocional e diminuição da autoestima ou que lhe prejudique e perturbe o pleno desenvolvimento ou que vise degradar ou controlar suas ações, comportamentos, crenças e decisões, mediante ameaça, constrangimento, humilhação, manipulação, isolamento, vigilância constante, perseguição contumaz, insulto, chantagem, violação de sua intimidade, ridicularização, exploração e limitação do direito de ir e vir ou qualquer outro meio que lhe cause prejuízo à saúde psicológica e à autodeterminação; (Inciso com redação dada pela Lei nº 13.772, de 19/12/2018) III - a violência sexual, entendida como qualquer conduta que a constranja a presenciar, a manter ou a participar

## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = """Use os trechos da lei maria da penha informados a seguir para responder a pergunta ao final.
Se você não souber a resposta ou não conseguir encontrar a resposta nos trechos dados, diga que não sabe responder, não invente uma resposta.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [21]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

/tmp/ipykernel_22952/336865589.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
/tmp/ipykernel_22952/336865589.py:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


Os tipos de violência contra a mulher são violência física, psicológica, sexual, patrimonial e moral.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [24]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

/tmp/ipykernel_22952/2983009556.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


Os tipos de violência contra a mulher incluem violência física, violência psicológica, violência sexual, violência 
patrimonial e violência moral.

In [25]:
import wandb
wandb.finish()